In [1]:
import os

from cjw.knowledgeqa.indexers.MarqoIndexer import MarqoIndexer
SERVER = 'http://localhost:8882'
TEST_INDEX = "test_indexer"

In [2]:
index = MarqoIndexer(SERVER, TEST_INDEX)

In [3]:
closestFacts = await index.search("What is M-137")
closestFacts

[{'id': '7751000',
  'text': "M-137 was a state trunkline highway in the US state of Michigan that served as a spur route to the Interlochen Center for the Arts and Interlochen State Park. It started south of the park and ran north between two lakes in the area and through the community of Interlochen to US Highway 31 (US 31) in Grand Traverse County. The highway was first shown without a number label on maps in 1930 and labeled after an extension the next year. The highway's current routing was established in the 1950s. Jurisdiction of the roadway was transferred from the Michigan Department of Transportation (MDOT) to the Grand Traverse County Road Commission in June 2020, and the highway designation was decommissioned in the process; signage was removed by August 2020 to reflect the changeover. ==Route description== M-137 began at the southern end of Interlochen State Park at an intersection with Vagabond Lane. Farther south, the roadway continues toward Green Lake Airport as County

In [4]:
def gatherFacts(facts):
    essentials = [
        f"[{f['_id']}]\n{f['text']}"
        for f in facts
    ]
    return "\n\n".join(essentials)

In [5]:
gatheredFacts = gatherFacts(closestFacts)
print(gatheredFacts)

[7751000]
M-137 was a state trunkline highway in the US state of Michigan that served as a spur route to the Interlochen Center for the Arts and Interlochen State Park. It started south of the park and ran north between two lakes in the area and through the community of Interlochen to US Highway 31 (US 31) in Grand Traverse County. The highway was first shown without a number label on maps in 1930 and labeled after an extension the next year. The highway's current routing was established in the 1950s. Jurisdiction of the roadway was transferred from the Michigan Department of Transportation (MDOT) to the Grand Traverse County Road Commission in June 2020, and the highway designation was decommissioned in the process; signage was removed by August 2020 to reflect the changeover. ==Route description== M-137 began at the southern end of Interlochen State Park at an intersection with Vagabond Lane. Farther south, the roadway continues toward Green Lake Airport as County Road 137 (CR 137), 

In [6]:
from cjw.utilities.llm.ChatPrompt import ChatPrompt

prompt = ChatPrompt(bot="assistant")

prompt.system(f"""
You will answer the user's questions based solly on the following facts.
Each fact are preceded by an ID in square brackets in a separate line before the fact.
Answer the questions by citing the ID in square brackets after your answer.
Say "I don't know" if nothing can be found from the facts.

For example:

Facts:
[12345a]
Pumps operate by some mechanism (typically reciprocating or rotary ), and consume energy to perform mechanical work by moving the fluid.
...


User:
    How do I remove water from my basement?

Assistant:
    You may try to use a pump.  [12345a]
""")

In [7]:
prompt.system(f"Facts:\n{gatheredFacts}", replace=False)

In [8]:
from cjw.utilities.llm.GptPortal import GptPortal

key = os.environ.get("OPENAI_API_KEY")
gpt = GptPortal.of(key)


In [9]:
prompt.show()

  0: (s) You will answer the user's que ... y try to use a pump. [12345a]
  1: (s) Facts: [7751000] M-137 was a s ... tegory:James Birrell buildings


In [10]:
prompt.user("Do you know any point of interests in Michigan?")
response = await gpt.chatCompletion(prompt.messages)
print(response)

{'role': 'assistant', 'content': "Yes, one point of interest in Michigan was the state trunkline highway, M-137. This highway served as a spur route to the Interlochen Center for the Arts and Interlochen State Park and was located in Michigan's Grand Traverse County. The highway was decommissioned and its signage removed in 2020. [7751000]", 'finish_reason': 'stop'}
